<a href="https://colab.research.google.com/github/Kangetheee/MLB_fan_interaction_bot/blob/main/MLB_trail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1️⃣ Authenticate & Install Dependencies

In [1]:
%pip install google-cloud-aiplatform gradio pandas google-cloud-storage firebase-admin --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.0 MB/s eta 0:00:00


# 2️⃣ Download the MLB Dataset from Google Cloud Storage (GCS)

In [3]:
from google.colab import auth
auth.authenticate_user()

# Ensure the correct project is set
PROJECT_ID = "mhproject01"
REGION = "us-central1"

# Set up Google Cloud SDK
!gcloud config set project {PROJECT_ID}
!gcloud auth application-default login

Updated property [core/project].

You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=VMqZKNxp2dLwBuDISSz

In [ ]:
!mkdir -p mlb-fan-content-interaction-data
!gsutil -m cp -r gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/* mlb-fan-content-interaction-data/

Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/2025-mlb-fan-favs-follows.json...
Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000000.json...
Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000001.json...
Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000002.json...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-000000000004.json...
Copying gs://gcp-mlb-hackathon-2025/datasets/mlb-fan-content-interaction-data/mlb-fan-content-interaction-data-0000

In [ ]:
!mv /content/mlb-fan-content-interaction-data /content/drive/MyDrive/mlb-data

# 3️⃣ Preprocess & Explore the Data

In [ ]:
import pandas as pd
import os
import json
from glob import glob

# Define the directory where the JSON files are stored
data_dir = "mlb-fan-content-interaction-data"

# List all JSON files in the directory
json_files = sorted(glob(os.path.join(data_dir, "*.json")))
print(f"Found {len(json_files)} JSON files.")

# Function to load and concatenate NDJSON files into a single DataFrame
def load_ndjson_to_dataframe(json_files):
    dataframes = []
    for file in json_files:
        with open(file, 'r') as f:
            # Read each line as a separate JSON object
            records = [json.loads(line) for line in f]
            # Convert list of dictionaries to DataFrame
            df = pd.json_normalize(records)  # Normalize nested JSON structures if needed
            dataframes.append(df)
    return pd.concat(dataframes, ignore_index=True)

# Load all NDJSON files into a single DataFrame
df = load_ndjson_to_dataframe(json_files)
print("\nCombined Data Preview:")
print(df.head())

Found 93 JSON files.


KeyboardInterrupt: 

# 4️⃣ Train a Simple Model (MLB Trivia & Fan Engagement Predictions)

In [ ]:
import vertexai
from vertexai.language_models import ChatModel

vertexai.init(project=PROJECT_ID, location=REGION)
chat_model = ChatModel.from_pretrained("gemini-1.5-flash-002")
chat = chat_model.start_chat()

# Test the model with a sample query
response = chat.send_message("What is your favorite MLB team?")
print("Gemini Pro Response:", response.text)

NameError: name 'PROJECT_ID' is not defined

In [ ]:
import firebase_admin
from firebase_admin import firestore

if not firebase_admin._apps:
    firebase_admin.initialize_app()

db = firestore.client()

def log_user_interaction(user_input, bot_response):
    """Logs user interactions to Firestore."""
    db.collection("chat_history").add({
        "user_input": user_input,
        "bot_response": bot_response,
        "timestamp": firestore.SERVER_TIMESTAMP
    })

# 5️⃣ Deploy a Gradio Chatbot with Vertex AI

In [ ]:
import gradio as gr

# Dummy function to fetch MLB stats (replace with real API calls)
def get_mlb_stats(player_name):
    return f"{player_name} hit 30 home runs last season!"

# Chatbot response function
def chatbot_response(user_input):
    if "stats" in user_input.lower():
        player_name = user_input.split()[-1]
        response = get_mlb_stats(player_name)
    else:
        response = chat.send_message(user_input).text

    # Log the interaction
    log_user_interaction(user_input, response)
    return response

# Launch the Gradio chatbot interface
chat_ui = gr.ChatInterface(chatbot_response)
chat_ui.launch(share=True)